In [ ]:
# Import necessary libraries

import torch
import torch.nn as nn
import torch.optim as optim
import math
import numpy as np
import pandas as pd
import os

# If GPU is available, set the device to GPU, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    torch.cuda.set_device(0)
    torch.set_default_device(torch.device("cuda:0"))
    print("Using GPU")

torch.cuda.empty_cache()

In [ ]:
# Load the vectorized conversation data

DATA_PROCESSED_PATH = os.path.join(os.getcwd(), '../data/processed')

seq_df = pd.read_csv(os.path.join(os.getcwd(), DATA_PROCESSED_PATH, "conversations_vectorized.csv"))

# Now convert from string to list
seq_df["seq1"] = seq_df["seq1"].apply(lambda x: [int(i) for i in x.split(",") if i.strip().isdigit()])
seq_df["seq2"] = seq_df["seq2"].apply(lambda x: [int(i) for i in x.split(",") if i.strip().isdigit()])

seq_df.head()

In [ ]:
# Define the Multi-Head Attention module

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        # Initialize dimensions
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        # Linear layers for transforming inputs
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Calculate attention scores
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        # Apply mask if provided (useful for preventing attention to certain parts like padding)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        
        # Softmax is applied to obtain attention probabilities
        attn_probs = torch.softmax(attn_scores, dim=-1)
        
        # Multiply by values to obtain the final output
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        # Reshape the input to have num_heads for multi-head attention
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        # Combine the multiple heads back to original shape
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        # Apply linear transformations and split heads
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        # Perform scaled dot-product attention
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        
        # Combine heads and apply output transformation
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [ ]:
# Define the FeedForward module

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(FeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [ ]:
# Define the Positional Encoding module

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [ ]:
# Define the Encoder and Decoder layers

class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = FeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x
    
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = FeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [ ]:
# Define the Transformer model

class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [ ]:
# Define the model parameters
src_vocab_size = 23570
tgt_vocab_size = 23570
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 464
dropout = 0.1

# Initialize the Transformer model
transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout).to(device)

# Define the optimizer and loss function
criterion = nn.CrossEntropyLoss(ignore_index=0).to(device)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

# Convert the sequences to tensors
seq1_tensor = torch.tensor(seq_df["seq1"].tolist(), dtype=torch.long).to(device)
seq2_tensor = torch.tensor(seq_df["seq2"].tolist(), dtype=torch.long).to(device)

# Split the data into training and validation sets
train_src_data = seq1_tensor[:150000]
train_tgt_data = seq2_tensor[:150000]
val_src_data = seq1_tensor[150000:]
val_tgt_data = seq2_tensor[150000:]

# Print the shapes of the training and validation data
print(f"Training source data shape: {train_src_data.shape}")
print(f"Training target data shape: {train_tgt_data.shape}")
print(f"Validation source data shape: {val_src_data.shape}")
print(f"Validation target data shape: {val_tgt_data.shape}")

In [ ]:
# Now let's train the model!

BATCH_SIZE = 16
NUM_EPOCHS = 100

transformer.train()

for epoch in range(NUM_EPOCHS):
    for batch in range(BATCH_SIZE):
        print(f"Epoch: {epoch+1}, batch: {batch+1}", end="\t")
        start_idx = batch * BATCH_SIZE
        end_idx = start_idx + BATCH_SIZE
        if end_idx > len(train_src_data):
            break
        
        train_src_batch = train_src_data[start_idx:end_idx]
        train_tgt_batch = train_tgt_data[start_idx:end_idx]

        # Forward pass
        optimizer.zero_grad()
        output = transformer(train_src_batch, train_tgt_batch[:, :-1])
        loss = criterion(output.contiguous().view(-1, tgt_vocab_size), train_tgt_batch[:, 1:].contiguous().view(-1))
        loss.backward()
        optimizer.step()
        print(f"Loss: {loss.item()}")

In [ ]:
# Validate the model after training

transformer.eval()

with torch.no_grad():
    for batch in range(BATCH_SIZE):
        print(f"Batch: {batch+1}", end="\t") 
        start_idx = batch * BATCH_SIZE
        end_idx = start_idx + BATCH_SIZE
        if end_idx > len(val_src_data):
            break

        val_src_batch = val_src_data[start_idx:end_idx]
        val_tgt_batch = val_tgt_data[start_idx:end_idx]

        val_output = transformer(val_src_batch, val_tgt_batch[:, :-1])
        val_loss = criterion(val_output.contiguous().view(-1, tgt_vocab_size), val_tgt_batch[:, 1:].contiguous().view(-1))

        print(f"Validation Loss: {val_loss.item()}")

In [ ]:
# Save the trained model

def save_model(model, filename):
    torch.save(model.state_dict(), filename)
    print(f"Model saved to {filename}")

save_model(transformer, "transformer_model.pth")